In [16]:
import pandas as pd
import os
import xml.etree.ElementTree as ET

In [17]:
# Definisci i percorsi dei file
csv_file_path = "../data/raw/ASLLRP/asllrp_sentence_signs_2025_06_28.csv"
base_xml_folder = "../data/raw/ASLLRP/full_videos_with_utterances"
output_csv_path = "../data/processed/utterances_with_translations.csv"

# Carica il file CSV
try:
    # Usa il motore 'python' e salta le righe malformate
    df = pd.read_csv(csv_file_path, sep=",", engine="python", on_bad_lines="skip")
    print("File CSV caricato con successo. Prime 5 righe:")
    # Rinomina la colonna
    if "Utterance video filename" in df.columns:
        df.rename(columns={"Utterance video filename": "video_name"}, inplace=True)
        print("Colonna 'Utterance video filename' rinominata in 'video_name'.")
    print(df.head())
except Exception as e:
    print(f"Errore durante il caricamento del file CSV: {e}")
    df = None

if df is None:
    print("Impossibile caricare il file.")

File CSV caricato con successo. Prime 5 righe:
Colonna 'Utterance video filename' rinominata in 'video_name'.
   Video ID number Main entry gloss label Entry/variant gloss label  \
0           340986                  IX-1p                     IX-1p   
1           340987                   HAVE                  (1h)HAVE   
2           340988                STRANGE                   STRANGE   
3           340989                  ABOUT                     ABOUT   
4           340990                SELF-1p                   SELF-1p   

  Occurrence label  Start frame of the sign video  \
0           IX-1p+                             34   
1         (1h)HAVE                             57   
2          STRANGE                             70   
3            ABOUT                            110   
4         SELF-1p+                            119   

   End frame of the sign video  Start frame of the containing utterance  \
0                           43                                       

In [18]:
if "df" in locals() and df is not None:
    print("Verifica dei duplicati...")
    # Controlla i duplicati basandosi sulle colonne che dovrebbero essere uniche per ogni video
    duplicate_rows = df[
        df.duplicated(subset=["video_name", "Source collection"], keep=False)
    ]
    print(
        f"Trovate {len(duplicate_rows)} righe che sono duplicati (basate su filename e source collection)."
    )
    if not duplicate_rows.empty:
        print("Esempio di righe duplicate:")
        # Mostra le prime 5 righe duplicate per dare un'idea
        print(duplicate_rows.head())
else:
    print("DataFrame non definito.")

Verifica dei duplicati...
Trovate 16759 righe che sono duplicati (basate su filename e source collection).
Esempio di righe duplicate:
   Video ID number Main entry gloss label Entry/variant gloss label  \
0           340986                  IX-1p                     IX-1p   
1           340987                   HAVE                  (1h)HAVE   
2           340988                STRANGE                   STRANGE   
3           340989                  ABOUT                     ABOUT   
4           340990                SELF-1p                   SELF-1p   

  Occurrence label  Start frame of the sign video  \
0           IX-1p+                             34   
1         (1h)HAVE                             57   
2          STRANGE                             70   
3            ABOUT                            110   
4         SELF-1p+                            119   

   End frame of the sign video  Start frame of the containing utterance  \
0                           43              

In [19]:
def get_translation(row):
    # Rimuovi .mp4 dal nome del file per ottenere l'ID
    utterance_id = str(row["video_name"]).replace(".mp4", "")
    source_collection = row["Source collection"]

    xml_folder_path = os.path.join(base_xml_folder, source_collection)

    if not os.path.isdir(xml_folder_path):
        return f"Cartella non trovata: {xml_folder_path}"

    xml_file_name = None
    for file in os.listdir(xml_folder_path):
        if file.startswith("xml") and file.endswith(".xml"):
            xml_file_name = file
            break

    if not xml_file_name:
        return "File XML non trovato nella cartella"

    xml_file_path = os.path.join(xml_folder_path, xml_file_name)

    try:
        tree = ET.parse(xml_file_path)
        root = tree.getroot()

        # Trova l'utterance con l'ID corrispondente
        for utterance in root.findall(".//UTTERANCE"):
            if utterance.get("ID") == utterance_id:
                translation_element = utterance.find("TRANSLATION")
                if translation_element is not None and translation_element.text:
                    # Rimuovi gli apici dall'inizio e dalla fine
                    return translation_element.text.strip("'")
                else:
                    return "Translation tag vuota"

    except ET.ParseError:
        return "Errore nel parsing del file XML"

    return "Traduzione non trovata"


# Applica la funzione per ottenere le traduzioni
if "df" in locals() and df is not None:
    print(f"Numero di righe prima della rimozione dei duplicati: {len(df)}")
    df.drop_duplicates(
        subset=["video_name", "Source collection"],
        keep="first",
        inplace=True,
    )
    print(f"Numero di righe dopo la rimozione dei duplicati: {len(df)}")

    df["caption"] = df.apply(get_translation, axis=1)

    # Crea il nuovo DataFrame con le colonne richieste
    output_df = df[["video_name", "Source collection", "caption"]]

    # Salva il DataFrame in un file CSV
    output_df.to_csv(output_csv_path, index=False)

    print(f"File CSV creato con successo in: {output_csv_path}")
    print("Prime 5 righe del nuovo file:")
    print(output_df.head())
else:
    print("DataFrame non definito, impossibile procedere.")

Numero di righe prima della rimozione dei duplicati: 16761
Numero di righe dopo la rimozione dei duplicati: 2127
File CSV creato con successo in: ../data/processed/utterances_with_translations.csv
Prime 5 righe del nuovo file:
      video_name      Source collection  \
0    2751812.mp4  Cory_2013-6-25_sc0106   
5   98592049.mp4  Cory_2013-6-25_sc0106   
17  98857459.mp4  Cory_2013-6-25_sc0106   
21  98881709.mp4  Cory_2013-6-25_sc0106   
29  98895873.mp4  Cory_2013-6-25_sc0106   

                                              caption  
0             There's something strange about myself.  
5                         I never lived in a house...  
17                             I was born in Florida.  
21  I lived there in a house with my parents and t...  
29  ... while I was growing up until I was six  an...  
File CSV creato con successo in: ../data/processed/utterances_with_translations.csv
Prime 5 righe del nuovo file:
      video_name      Source collection  \
0    2751812.mp4  Cor